# 05 - Validation
CV simple + m?tricas.

In [ ]:
import os, sys, json

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir(PROJECT_ROOT)
else:
    PROJECT_ROOT = os.getcwd()

sys.path.insert(0, PROJECT_ROOT)

In [ ]:
from src.io_utils import read_csv_robust, standardize_columns, coerce_numeric
from src.validation import simple_cross_validation

with open("config/project.json", "r", encoding="utf-8") as f:
    cfg = json.load(f)

if not os.path.exists(cfg["data_csv_path"]):
    from src.make_demo_data import make_demo_csv
    make_demo_csv(cfg["data_csv_path"])

df = read_csv_robust(cfg["data_csv_path"])

mapping = {
    "x": cfg["columns"].get("x"),
    "y": cfg["columns"].get("y"),
    "z": cfg["columns"].get("z"),
    "var": cfg["columns"].get("variable_objetivo"),
}

df = standardize_columns(df, mapping)
coerce_numeric(df, ["x", "y", "z", "var"])
df.dropna(subset=["x", "y", "var"], inplace=True)

cv_df, metrics = simple_cross_validation(
    df,
    "x",
    "y",
    "var",
    radius=float(cfg["kriging"].get("search_radius", 150.0)),
    max_samples=int(cfg["kriging"].get("max_samples", 12)),
)

print(metrics)